In [1]:
# 분석
import pandas as pd
import numpy as np

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

# ML
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score,precision_score,recall_score, roc_curve, classification_report,precision_recall_curve
from sklearn.metrics import make_scorer
from xgboost.sklearn import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

### 코랩이용시

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
d_path = '/content/drive/MyDrive/Colab Notebooks/Dacon/'
X_train = pd.read_csv(d_path + 'train_features.csv')
y_train = pd.read_csv(d_path + 'train_labels.csv')
X_test = pd.read_csv(d_path + 'test_features.csv')
submission = pd.read_csv(d_path + 'sample_submission.csv')

display(X_train.head(3))
display(y_train.head(3))
display(X_test.head(3))

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629


,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)


,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,3125,0,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668
1,3125,1,-0.462548,0.012462,-0.053726,56.953059,96.185341,16.278458
2,3125,2,-0.363481,-0.091789,-0.130004,29.557396,93.836453,13.329043


### 쥬피터 이용시

In [ ]:
# X_train = pd.read_csv('train_features.csv')
# y_train = pd.read_csv( 'train_labels.csv')
# X_test = pd.read_csv('test_features.csv')
# submission = pd.read_csv('sample_submission.csv')

----------------------------

#### X_train의 데이터를 통해서 id와 time을 제외한 피쳐를 대상으로 id를 기준으로 잡아 합, 평균, 중앙값, 최소값, 최대값을 구함

In [4]:
X_pivot_train = pd.pivot_table(data = X_train, # X_train의 데이터를 통해서
                               values = X_train.columns[2:],  # id와 time을 제외한 피쳐를 대상으로
                               index = 'id', # id를 기준으로 잡아
                               aggfunc = ['sum','mean',         # 합, 평균
                                          'median','min','max', # 중앙값 최소값, 최대값
                                          'std','var'           # 베셀 보정 표본 표준편차, 비편향 편차 의 값을 구합니다.
                                         ]
                              )

X_pivot_test = pd.pivot_table(data = X_test, 
                               values = X_test.columns[2:], 
                               index = 'id', 
                               aggfunc = ['sum','mean',        
                                          'median','min','max',
                                          'std','var'          
                                         ]
                              )

display(X_pivot_train.head(3))
display(X_pivot_test.head(3))

sum                          ...          var                           
         acc_x       acc_y       acc_z  ...         gy_x          gy_y         gy_z
id                                      ...                                        
0   558.797337 -131.082711 -222.252919  ...   176.470384    590.513292   638.834979
1  -459.948117 -190.354639   -2.534051  ...  6279.700472   9217.015511  5707.098884
2    23.901616  -49.441742  375.607013  ...   646.325142  14150.683677   193.775778

[3 rows x 42 columns]

sum                          ...          var                          
           acc_x       acc_y       acc_z  ...         gy_x         gy_y         gy_z
id                                        ...                                       
3125 -611.238360  -11.744605 -139.355669  ...   166.357553  1023.553453   150.102867
3126 -313.705824  367.296809  -42.655405  ...  2665.150566  2089.066820  3795.159662
3127  304.167948  542.291164  -84.658968  ...   518.511372   181.383940   530.909012

[3 rows x 42 columns]

In [5]:
X_pivot_train.columns[:5]

MultiIndex([('sum', 'acc_x'),
            ('sum', 'acc_y'),
            ('sum', 'acc_z'),
            ('sum',  'gy_x'),
            ('sum',  'gy_y')],
           )

In [6]:
X_columns = [agg + '_' + column for agg,column in X_pivot_train.columns]
X_pivot_train.columns = X_columns
X_pivot_test.columns = X_columns
display(X_pivot_train.head())

,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,mean_gy_x,mean_gy_y,mean_gy_z,median_acc_x,median_acc_y,median_acc_z,median_gy_x,median_gy_y,median_gy_z,min_acc_x,min_acc_y,min_acc_z,min_gy_x,min_gy_y,min_gy_z,max_acc_x,max_acc_y,max_acc_z,max_gy_x,max_gy_y,max_gy_z,std_acc_x,std_acc_y,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,0.931329,-0.218471,-0.370422,-1.865269,-3.359506,1.182107,0.956149,-0.240638,-0.346749,-1.273569,-2.362230,1.913286,0.591940,-0.624113,-0.786336,-46.254836,-85.887677,-79.930029,1.344268,0.176871,-0.054876,31.644123,69.847244,55.953827,0.191479,0.177131,0.135131,13.284216,24.300479,25.275185,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979
1,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,-0.766580,-0.317258,-0.004223,11.071600,1.740475,1.393294,-0.805767,-0.228905,-0.034583,3.810650,8.043707,-0.655819,-2.156208,-1.295598,-1.019531,-325.328531,-315.096003,-270.980823,1.234020,0.700065,0.888661,286.624363,389.608060,340.170199,0.495528,0.336415,0.499395,79.244561,96.005289,75.545343,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884
2,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.039836,-0.082403,0.626012,-8.472951,0.597877,3.053291,0.140667,-0.062598,0.634781,-8.112557,19.306132,3.568888,-1.142847,-0.690990,0.073846,-164.779067,-249.953944,-44.192071,1.219836,0.650645,1.332992,73.525082,297.320834,55.642836,0.711972,0.147127,0.248807,25.422926,118.956646,13.920337,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778
3,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,-0.887702,-0.087668,0.227357,17.744167,4.800931,-5.869898,-0.880343,-0.054577,0.231537,8.229938,1.783260,-3.853078,-1.417751,-0.540827,-0.257124,-69.419166,-82.537304,-85.600536,-0.622250,0.283721,0.598814,192.765368,159.083788,56.456908,0.130899,0.194008,0.205882,42.928860,36.953466,23.647153,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841
4,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,-0.659018,-0.337067,0.202758,-4.819638,9.651713,4.453382,-0.941146,-0.168467,0.293556,-1.292194,0.977772,-0.750283,-2.429109,-2.055076,-1.250483,-769.076518,-243.909948,-270.581913,0.599720,1.724782,2.678034,613.972600,284.952954,221.015193,0.495170,0.570305,0.389646,108.258866,60.514531,46.148326,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017


In [7]:
X_pivot_train = X_pivot_train.reset_index()
X_pivot_test = X_pivot_test.reset_index()

display(X_pivot_train.head())
display(X_pivot_test.head())

,id,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,mean_gy_x,mean_gy_y,mean_gy_z,median_acc_x,median_acc_y,median_acc_z,median_gy_x,median_gy_y,median_gy_z,min_acc_x,min_acc_y,min_acc_z,min_gy_x,min_gy_y,min_gy_z,max_acc_x,max_acc_y,max_acc_z,max_gy_x,max_gy_y,max_gy_z,std_acc_x,std_acc_y,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
0,0,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,0.931329,-0.218471,-0.370422,-1.865269,-3.359506,1.182107,0.956149,-0.240638,-0.346749,-1.273569,-2.362230,1.913286,0.591940,-0.624113,-0.786336,-46.254836,-85.887677,-79.930029,1.344268,0.176871,-0.054876,31.644123,69.847244,55.953827,0.191479,0.177131,0.135131,13.284216,24.300479,25.275185,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979
1,1,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,-0.766580,-0.317258,-0.004223,11.071600,1.740475,1.393294,-0.805767,-0.228905,-0.034583,3.810650,8.043707,-0.655819,-2.156208,-1.295598,-1.019531,-325.328531,-315.096003,-270.980823,1.234020,0.700065,0.888661,286.624363,389.608060,340.170199,0.495528,0.336415,0.499395,79.244561,96.005289,75.545343,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884
2,2,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.039836,-0.082403,0.626012,-8.472951,0.597877,3.053291,0.140667,-0.062598,0.634781,-8.112557,19.306132,3.568888,-1.142847,-0.690990,0.073846,-164.779067,-249.953944,-44.192071,1.219836,0.650645,1.332992,73.525082,297.320834,55.642836,0.711972,0.147127,0.248807,25.422926,118.956646,13.920337,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778
3,3,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,-0.887702,-0.087668,0.227357,17.744167,4.800931,-5.869898,-0.880343,-0.054577,0.231537,8.229938,1.783260,-3.853078,-1.417751,-0.540827,-0.257124,-69.419166,-82.537304,-85.600536,-0.622250,0.283721,0.598814,192.765368,159.083788,56.456908,0.130899,0.194008,0.205882,42.928860,36.953466,23.647153,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841
4,4,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,-0.659018,-0.337067,0.202758,-4.819638,9.651713,4.453382,-0.941146,-0.168467,0.293556,-1.292194,0.977772,-0.750283,-2.429109,-2.055076,-1.250483,-769.076518,-243.909948,-270.581913,0.599720,1.724782,2.678034,613.972600,284.952954,221.015193,0.495170,0.570305,0.389646,108.258866,60.514531,46.148326,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017


,id,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,mean_gy_x,mean_gy_y,mean_gy_z,median_acc_x,median_acc_y,median_acc_z,median_gy_x,median_gy_y,median_gy_z,min_acc_x,min_acc_y,min_acc_z,min_gy_x,min_gy_y,min_gy_z,max_acc_x,max_acc_y,max_acc_z,max_gy_x,max_gy_y,max_gy_z,std_acc_x,std_acc_y,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
0,3125,-611.238360,-11.744605,-139.355669,-1911.076959,1639.123438,-1200.410049,-1.018731,-0.019574,-0.232259,-3.185128,2.731872,-2.000683,-1.064222,-0.005735,-0.268442,-3.770150,0.108956,-1.607847,-1.564000,-0.470937,-0.573836,-50.429364,-81.607713,-35.446915,-0.275446,0.228040,0.286182,56.953059,96.185341,49.981455,0.236232,0.091641,0.174672,12.897967,31.993022,12.251648,0.055806,0.008398,0.030510,166.357553,1023.553453,150.102867
1,3126,-313.705824,367.296809,-42.655405,-10644.915365,4184.863263,-2162.747150,-0.522843,0.612161,-0.071092,-17.741526,6.974772,-3.604579,-0.677411,0.606215,-0.026089,-14.305258,-0.974696,-10.833508,-1.929033,-0.200678,-1.212052,-273.572486,-97.100707,-147.597574,0.627571,1.708743,0.671876,132.830402,241.240196,169.417650,0.539688,0.333015,0.383800,51.625096,45.706311,61.604867,0.291264,0.110899,0.147302,2665.150566,2089.066820,3795.159662
2,3127,304.167948,542.291164,-84.658968,-1307.846921,-1350.871152,-235.904841,0.506947,0.903819,-0.141098,-2.179745,-2.251452,-0.393175,0.497370,0.931239,-0.156647,0.476247,-2.319172,-3.087735,-0.792916,0.219008,-0.484614,-99.799971,-98.420987,-154.477074,2.972063,1.941820,0.644154,160.426058,74.530763,97.211730,0.219934,0.191485,0.152077,22.770845,13.467885,23.041463,0.048371,0.036666,0.023128,518.511372,181.383940,530.909012
3,3128,-346.561617,-366.333946,18.891749,485.147442,-1790.981310,-14.590798,-0.577603,-0.610557,0.031486,0.808579,-2.984969,-0.024318,-0.880541,-0.507927,-0.092860,1.457625,-0.269600,-0.404583,-1.045889,-1.294482,-0.469924,-229.072919,-168.031080,-117.297766,0.337281,-0.258476,0.702574,119.527887,118.268797,167.860762,0.431713,0.233601,0.326569,42.818157,45.069932,37.967372,0.186376,0.054569,0.106647,1833.394532,2031.298793,1441.521366
4,3129,-443.184021,109.521180,240.781103,-858.922755,865.419381,3447.298941,-0.738640,0.182535,0.401302,-1.431538,1.442366,5.745498,-0.703842,0.122280,0.432678,-3.066063,1.631638,1.866352,-2.153047,-0.860883,-0.631258,-345.447240,-223.475411,-125.598600,0.015642,1.562602,1.037876,366.167357,226.728939,138.130133,0.305797,0.314294,0.261848,92.301963,67.911174,43.353007,0.093512,0.098781,0.068565,8519.652350,4611.927587,1879.483194


In [8]:
X_pivot_train.shape

(3125, 43)

### 학습에 사용할 데이터

In [9]:
ftr = X_pivot_train
target = y_train['label']

### LGBM

In [10]:
X_train, X_val, y_train, y_val = train_test_split(ftr, target, test_size=0.2, random_state=156)

lgbm_wrapper = LGBMClassifier(n_estimators=1000, num_leaves=100, subsample=0.8,
                      min_child_samples=320, max_depth=30)

evals = [(ftr, target)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss', eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(X_pivot_test)
pred_proba = lgbm_wrapper.predict_proba(X_pivot_test)[:,:]

[1]	valid_0's multi_logloss: 2.54967	valid_0's multi_logloss: 2.54967
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 2.44754	valid_0's multi_logloss: 2.44754
[3]	valid_0's multi_logloss: 2.36224	valid_0's multi_logloss: 2.36224
[4]	valid_0's multi_logloss: 2.28827	valid_0's multi_logloss: 2.28827
[5]	valid_0's multi_logloss: 2.22081	valid_0's multi_logloss: 2.22081
[6]	valid_0's multi_logloss: 2.16124	valid_0's multi_logloss: 2.16124
[7]	valid_0's multi_logloss: 2.10861	valid_0's multi_logloss: 2.10861
[8]	valid_0's multi_logloss: 2.0579	valid_0's multi_logloss: 2.0579
[9]	valid_0's multi_logloss: 2.00773	valid_0's multi_logloss: 2.00773
[10]	valid_0's multi_logloss: 1.96496	valid_0's multi_logloss: 1.96496
[11]	valid_0's multi_logloss: 1.92285	valid_0's multi_logloss: 1.92285
[12]	valid_0's multi_logloss: 1.88298	valid_0's multi_logloss: 1.88298
[13]	valid_0's multi_logloss: 1.84782	valid_0's multi_logloss: 1.84782
[14]	valid_0's multi_logl

In [11]:
print(pred_proba.shape)

(782, 61)


# 앙상블 Voting을 위한 작업

---------------------------------------
### RandomForest 최적 Parameter 찾기

In [12]:
def SCORES(y_val, pred, proba) :
    #acc = accuracy_score(y_val, pred)
    f1 = f1_score(y_val, pred, average='macro')
    #print(y_val[:10])
    #print(proba[:10])
    auc = roc_auc_score(y_val, proba, average='macro', multi_class='ovo')

In [13]:
print(y_val.shape)
print(pred_proba.shape)
print(preds.shape)

(625,)
(782, 61)
(782,)


In [25]:
rf_model = RandomForestClassifier(n_jobs=-1, random_state=0, min_samples_leaf=30)

In [26]:
# 수정 필요
my_hyper_param = {
    "n_estimators"      :[700,800], # 700 
    "max_depth"         :[15,17,19],  # 17
    "min_samples_leaf"  :[1,3]  # 1
}

In [43]:
from sklearn.metrics import make_scorer

gcv_model = GridSearchCV(rf_model, param_grid=my_hyper_param, scoring='roc_auc_ovr', refit=True, cv=5, verbose=0)
#scoring=make_scorer(recall_score, average='micro')

gcv_model.fit(ftr, target)

print("best_estimator:", gcv_model.best_estimator_)
print("best_params:",    gcv_model.best_params_)
print("best_score:" ,    gcv_model.best_score_)

KeyboardInterrupt: ignored

### RandomForest 학습

In [44]:
rfc = rf_model = RandomForestClassifier(n_jobs=-1, random_state=0, min_samples_leaf=1, n_estimators=700, max_depth=17)
rfc.fit(ftr, target)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=17, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

### XGBOOST 최적 Parameter 찾기

In [ ]:
estimator = XGBClassifier()
 
parameters = {
              'objective':['binary:logistic'],
              'learning_rate': [0.05, 0.1], #so called `eta` value 0.05
              'max_depth': [6,8,10],    # 8
              'n_estimators': [300,350,400],    # 350 
              'seed': [1337]}

clf = GridSearchCV(estimator, parameters, n_jobs=5, 
                   cv=5, 
                   scoring='roc_auc_ovr',
                   verbose=2, refit=True)
 
clf.fit(ftr, target)
print('Best parameters found by grid search are:', clf.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed: 38.1min
[Parallel(n_jobs=5)]: Done  90 out of  90 | elapsed: 85.0min finished


Best parameters found by grid search are: {'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 350, 'objective': 'binary:logistic', 'seed': 1337}


### XGBOOST 모델

In [45]:
estimator = XGBClassifier()
xgb = XGBClassifier(colsample_bytree= 0.7, learning_rate= 0.05, max_depth= 8, min_child_weight= 11, missing= -999, n_estimators= 350, nthread= 4, objective= 'binary:logistic', seed= 1337, silent= 1, subsample= 0.8)
xgb.fit(ftr, target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=8,
              min_child_weight=11, missing=-999, n_estimators=350, n_jobs=1,
              nthread=4, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
              silent=1, subsample=0.8, verbosity=1)

### LGBM 최적 Parameter 찾기

In [ ]:
estimator = LGBMClassifier(num_leaves=2)
param_grid = {
    'learning_rate': [0.1, 0.05, 0.5],
    'n_estimators': [1100]    # 1100 이상 확인 필요
}

grid = GridSearchCV(estimator, param_grid, cv=5, scoring='roc_auc_ovr')

grid.fit(ftr, target)

print('Best parameters found by grid search are:', grid.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.05, 'n_estimators': 1100}


### LGBM 모델

In [46]:
lgb = LGBMClassifier(n_estimators=1100, num_leaves=100, subsample=0.8,
                      min_child_samples=320, max_depth=30, learning_rate=0.05)

evals = [(ftr, target)]
lgb.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss', eval_set=evals, verbose=True)

[1]	valid_0's multi_logloss: 2.68286	valid_0's multi_logloss: 2.68286
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 2.61883	valid_0's multi_logloss: 2.61883
[3]	valid_0's multi_logloss: 2.56097	valid_0's multi_logloss: 2.56097
[4]	valid_0's multi_logloss: 2.50912	valid_0's multi_logloss: 2.50912
[5]	valid_0's multi_logloss: 2.46081	valid_0's multi_logloss: 2.46081
[6]	valid_0's multi_logloss: 2.41518	valid_0's multi_logloss: 2.41518
[7]	valid_0's multi_logloss: 2.37265	valid_0's multi_logloss: 2.37265
[8]	valid_0's multi_logloss: 2.33277	valid_0's multi_logloss: 2.33277
[9]	valid_0's multi_logloss: 2.29567	valid_0's multi_logloss: 2.29567
[10]	valid_0's multi_logloss: 2.26031	valid_0's multi_logloss: 2.26031
[11]	valid_0's multi_logloss: 2.22525	valid_0's multi_logloss: 2.22525
[12]	valid_0's multi_logloss: 2.19131	valid_0's multi_logloss: 2.19131
[13]	valid_0's multi_logloss: 2.15924	valid_0's multi_logloss: 2.15924
[14]	valid_0's multi_lo

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.05, max_depth=30,
               min_child_samples=320, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=1100, n_jobs=-1, num_leaves=100,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=0.8, subsample_for_bin=200000,
               subsample_freq=0)

### 앙상블 Voting

In [47]:
from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[('rfc', rfc), ('lgb', lgb), ('xgb', xgb)], voting='soft')
eclf.fit(ftr, target)
y_prob=eclf.predict_proba(X_pivot_test)

-------------------------------------
### Stacking(CV)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [42]:
rf_model = RandomForestClassifier()
svc_model = SVC(probability=True)
lr_model = LogisticRegression()

rf_train_fold_predict  = np.zeros((X_train.shape[0], 1))
svc_train_fold_predict = np.zeros((X_train.shape[0], 1))
lr_train_fold_predict  = np.zeros((X_train.shape[0], 1))

rf_test_predict  = np.zeros((X_val.shape[0], 5))
svc_test_predict = np.zeros((X_val.shape[0], 5))
lr_test_predict  = np.zeros((X_val.shape[0], 5))

In [22]:
skFold = StratifiedKFold(n_splits=5, random_state=111, shuffle=True)
for loop_cnt, (train_fold_idx, val_fold_idx) in enumerate(skFold.split(X_train, y_train)):
    X_train_fold = X_train.iloc[train_fold_idx]
    y_train_fold = y_train.iloc[train_fold_idx]
    X_val_fold   = X_train.iloc[val_fold_idx]
    y_val_fold   = y_train.iloc[val_fold_idx]
    #--------------------------------------------------------- fit    X_train_fold, y_train_fold
    rf_model.fit(X_train_fold, y_train_fold)
    svc_model.fit(X_train_fold, y_train_fold)
    lr_model.fit(X_train_fold, y_train_fold)
    # --------------------------------------------------------- predict  X_val_fold
    rf_pred_fold = rf_model.predict(X_val_fold)
    svc_pred_fold = svc_model.predict(X_val_fold)
    lr_pred_fold = lr_model.predict(X_val_fold)
    print(loop_cnt, rf_pred_fold.shape,  svc_pred_fold.shape, lr_pred_fold.shape)

    rf_train_fold_predict[val_fold_idx , :]   = rf_pred_fold.reshape(-1, 1)
    svc_train_fold_predict[val_fold_idx , :]  = svc_pred_fold.reshape(-1, 1)
    lr_train_fold_predict[val_fold_idx , :]   = lr_pred_fold.reshape(-1, 1)
    # --------------------------------------------------------- predict   X_val
    rf_pred_val  = rf_model.predict(X_val)
    svc_pred_val = svc_model.predict(X_val)
    lr_pred_val  = lr_model.predict(X_val)

    rf_test_predict[:, loop_cnt]  = rf_pred_val
    svc_test_predict[:, loop_cnt] = svc_pred_val
    lr_test_predict[:, loop_cnt]  = lr_pred_val

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0 (500,) (500,) (500,)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


1 (500,) (500,) (500,)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2 (500,) (500,) (500,)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


3 (500,) (500,) (500,)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


4 (500,) (500,) (500,)


In [23]:
rf_test_predict_mean = np.mean(rf_test_predict, axis=1)
print(rf_test_predict_mean.shape)
rf_test_predict_mean = rf_test_predict_mean.reshape(-1, 1)
print(rf_test_predict_mean.shape)

svc_test_predict_mean = np.mean(svc_test_predict, axis=1)
print(svc_test_predict_mean.shape)
svc_test_predict_mean = svc_test_predict_mean.reshape(-1, 1)
print(svc_test_predict_mean.shape)

lr_test_predict_mean = np.mean(lr_test_predict, axis=1)
print(lr_test_predict_mean.shape)
lr_test_predict_mean = lr_test_predict_mean.reshape(-1, 1)
print(lr_test_predict_mean.shape)

(625,)
(625, 1)
(625,)
(625, 1)
(625,)
(625, 1)


In [41]:
new_train_data = np.concatenate([rf_train_fold_predict, svc_train_fold_predict, lr_train_fold_predict], axis=1)
new_test_mean = np.concatenate([rf_test_predict_mean, svc_test_predict_mean, lr_test_predict_mean], axis=1)
print(new_train_data.shape)
print(new_test_mean.shape)
print(y_train.shape)
print(X_pivot_test.shape)

(2500, 3)
(625, 3)
(2500,)
(782, 43)


In [39]:
from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[('rfc', rfc), ('lgb', lgb), ('xgb', xgb)], voting='soft')
eclf.fit(new_train_data, y_train)
y_prob=eclf.predict_proba(X_pivot_test)

ValueError: ignored

In [48]:
y_prob.shape

(782, 61)

In [49]:
submission.iloc[:,1:] = y_prob
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.002281,0.000375,0.002372,0.000566,0.002329,0.000419,0.001476,0.000660,0.000524,0.035807,0.166250,0.153824,0.010306,0.327352,0.030346,0.002031,0.000256,0.000728,0.000366,0.001673,0.001885,0.000974,0.001426,0.002180,0.094302,0.000216,0.007972,0.001175,0.000303,0.000696,0.005671,0.004196,0.004254,0.000487,0.002615,0.000482,0.018254,0.001776,0.001479,0.000935,0.000229,0.000370,0.002965,0.000682,0.000330,0.001800,0.000495,0.000329,0.000948,0.002129,0.000355,0.020284,0.004387,0.013174,0.002137,0.000314,0.000808,0.001663,0.051162,0.000631,0.003587
1,3126,0.001849,0.000107,0.000159,0.000433,0.001248,0.000082,0.000173,0.000653,0.000360,0.000933,0.000914,0.000435,0.000107,0.000261,0.000084,0.000283,0.000542,0.000065,0.000036,0.000068,0.000128,0.000743,0.004771,0.002062,0.000798,0.000053,0.953555,0.000157,0.000029,0.000126,0.000356,0.001352,0.000440,0.001554,0.001870,0.000196,0.000533,0.000068,0.000082,0.000313,0.000587,0.000272,0.001498,0.000413,0.000185,0.000096,0.000034,0.000545,0.000513,0.002247,0.003334,0.000334,0.000101,0.000197,0.000160,0.000051,0.000156,0.005416,0.000250,0.000083,0.005554
2,3127,0.004643,0.431493,0.000797,0.002265,0.000556,0.000609,0.035973,0.005967,0.008118,0.001156,0.002466,0.000876,0.002041,0.001523,0.000375,0.000491,0.014189,0.016464,0.000431,0.000943,0.000790,0.001029,0.000863,0.000257,0.000822,0.000645,0.053979,0.020275,0.005963,0.011627,0.000470,0.000197,0.023941,0.017108,0.003851,0.007820,0.016216,0.013400,0.000566,0.004675,0.010579,0.005360,0.002270,0.009730,0.004191,0.049525,0.000482,0.013466,0.023672,0.018723,0.001695,0.000895,0.001879,0.002254,0.024444,0.001278,0.021623,0.000388,0.001307,0.081587,0.008781
3,3128,0.001662,0.000829,0.002819,0.000133,0.000454,0.000098,0.000036,0.002242,0.000279,0.000324,0.000409,0.000386,0.000277,0.000255,0.000036,0.000306,0.002342,0.000146,0.000091,0.000138,0.000219,0.007578,0.002604,0.000891,0.000489,0.000038,0.940939,0.000077,0.000194,0.000128,0.001255,0.000324,0.000704,0.003928,0.001531,0.000125,0.001041,0.000100,0.000061,0.000125,0.000072,0.001211,0.002211,0.004189,0.000094,0.000062,0.000196,0.000144,0.000728,0.003102,0.003246,0.000075,0.000196,0.000315,0.000049,0.000158,0.000050,0.001574,0.000083,0.000108,0.006522
4,3129,0.004006,0.000072,0.000057,0.000234,0.001843,0.000173,0.000060,0.000183,0.000592,0.000390,0.000519,0.000341,0.000083,0.000197,0.000153,0.000602,0.000318,0.000071,0.000209,0.000061,0.000155,0.000556,0.007587,0.001465,0.000230,0.000022,0.952730,0.000020,0.000051,0.000027,0.000265,0.000829,0.001014,0.000513,0.005689,0.000111,0.000437,0.000025,0.000090,0.000057,0.000059,0.000191,0.000473,0.000382,0.000047,0.000049,0.000031,0.000049,0.000437,0.002001,0.005541,0.000113,0.000084,0.000134,0.000015,0.000044,0.000060,0.001627,0.000067,0.000055,0.006507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.009399,0.000165,0.000100,0.000631,0.054618,0.000599,0.000116,0.000211,0.000625,0.003177,0.001422,0.004069,0.000599,0.001076,0.001382,0.006566,0.000450,0.000734,0.003547,0.000459,0.000836,0.000375,0.008855,0.000702,0.000636,0.000214,0.843901,0.000140,0.000244,0.000086,0.002277,0.002611,0.002302,0.000718,0.011577,0.000996,0.002036,0.000150,0.000138,0.000206,0.002218,0.000286,0.001307,0.000721,0.000427,0.000078,0.000192,0.000115,0.001948,0.002547,0.006170,0.005210,0.000456,0.002484,0.000059,0.000081,0.000715,0.002111,0.000327,0.000683,0.002923
778,3903,0.002697,0.000142,0.000072,0.000162,0.002334,0.000099,0.000029,0.000119,0.000351,0.000403,0.000435,0.000311,0.000066,0.000153,0.000149,0.001114,0.000519,0.000072,0.000905,0.000055,0.000776,0.000503,0.004447,0.001510,0.000312

In [50]:
submission.to_csv(d_path + 'baseline_rf1.csv', index=False)